In [3]:
import torch
import time
# Создайте большие матрицы размеров:
x = torch.rand((64,1024,1024))# - 64 x 1024 x 1024
y = torch.rand((128,512,512))# - 128 x 512 x 512
z = torch.rand((256,256,256))# - 256 x 256 x 256
# Заполните их случайными числами

In [33]:
# Создайте функцию для измерения времени выполнения операций
# Используйте torch.cuda.Event() для точного измерения на GPU
# Используйте time.time() для измерения на CPU
import torch
import time

def measure_performance(func, *args):
    gpu = torch.device("cuda")
    
    start_time = time.time()
    func(*args)
    cpu_time = (time.time() - start_time) * 1000
    
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    gpu_args = [arg.to(gpu) if isinstance(arg, torch.Tensor) else arg for arg in args]
    
    torch.cuda.synchronize()
    start_event.record()
    func(*gpu_args)
    end_event.record()
    torch.cuda.synchronize()
    
    gpu_time = start_event.elapsed_time(end_event)

    return cpu_time,gpu_time


In [34]:
# Сравните время выполнения следующих операций на CPU и CUDA:
print("Операция          | CPU (мс) | GPU (мс) | Ускорение(x)")
matmul = measure_performance(torch.matmul, x,x)# - Матричное умножение (torch.matmul)
print("Матричное умножение|",matmul[0],"|",matmul[1],"|",matmul[0]/matmul[1])
add = measure_performance(torch.add, x,x)# - Поэлементное сложение
print("Поэлементное сложение|",add[0],"|",add[1],"|",add[0]/add[1])
mul = measure_performance(torch.mul, x,x)# - Поэлементное умножение
print("Поэлементное умножение|",mul[0],"|",mul[1],"|",mul[0]/mul[1])
transpose = measure_performance(torch.transpose, x, 0,1)# - Транспонирование
print("Транспонирование|",transpose[0],"|",transpose[1],"|",transpose[0]/transpose[1])
summ = measure_performance(torch.sum, x)# - Вычисление суммы всех элементов
print("Вычисление суммы всех элементов|",summ[0],"|",summ[1],"|",summ[0]/summ[1])


# Для каждой операции:
# 1. Измерьте время на CPU
# 2. Измерьте время на GPU (если доступен)
# 3. Вычислите ускорение (speedup)
# 4. Выведите результаты в табличном виде

Операция          | CPU (мс) | GPU (мс) | Ускорение
Матричное умножение| 1241.624116897583 | 22.88412857055664 | 54.25699794813648 x
Поэлементное сложение| 25.067806243896484 | 2.0959999561309814 | 11.959831473550837 x
Поэлементное умножение| 24.17612075805664 | 2.1234560012817383 | 11.385270400452708 x
Транспонирование| 0.016450881958007812 | 0.08902399986982346 | 0.18479153915869131 x
Вычисление суммы всех элементов| 7.637977600097656 | 0.7407680153846741 | 10.31089010522589 x


In [ ]:
# Проанализируйте результаты:
# - Какие операции получают наибольшее ускорение на GPU?
#Операции связанные с арифметическими операциями (умножение, сложение)
# - Почему некоторые операции могут быть медленнее на GPU?
#В моем случае единственная операция выполнившаяся хуже на gpu - транспонирование, потому что это операция в которой приходится много работать с памятью, а это происходит эффективнее на CPU.
# - Как размер матриц влияет на ускорение?
#Чем больше размер матриц, тем больше ускорение.
# - Что происходит при передаче данных между CPU и GPU?
#Данные отправляются из оперативной памяти в видеопамять или обратно, где с ней затем работает соответствующее устройство